### Extração do Modelo de Data Mining
Para a previsão do ESRB rating de cada jogo a carregar no Data Warehouse foi utilizado o algoritmo com melhor exatidão identificado pela equipa na fase de Evaluation, o MLP Classifier.
<br>
<br>Uma vez que nos foi entregue um dataset com as datas de lançamento dos jogos, o nosso cliente (Sony) pediu para prever todos os jogos com data de lançamento igual ou superior a 2019, sendo os restantes para treino do modelo. A partir do SweetViz conseguimos afirmar que os novos dados de treino e teste representam, ordenadamente, 62% e 38% do dataset (train.csv). Devido a isto, o valor de exatidão alterou de 87,54% para 85.19%, porém permanece no mesmo nível que o anterior (exatidão entre 85%-89%).
<br>
<br>Posto isto, treinamos o modelo e extraímos o mesmo com o auxílio da libraria joblib de Python para o ficheiro “MLP_Classifier.pkl”. Porém fizemos o mesmo para os outros modelos, para os utilizar no nosso previsor online, assim o utilizador pode escolher entre os diversos modelos estudados qual pretende utilizar para prever o rating do seu jogo.
<br>
<br>Com o modelo extraído, no passo transformation do ETL, apenas necessitamos de carregar o ficheiro do modelo e invocar o método predict com as features do cenário 2 (ordenadas pela ordem de treino) como parâmetros.

In [1]:
import pandas as pd

# Models
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.svm import NuSVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

from imblearn.over_sampling import SMOTE

# Extract Trained Model
import joblib

import warnings
warnings.filterwarnings("ignore")

### Load Variables

In [2]:
# Models (Dictionary)
state = 0
models = {
    "SGD_Classifier":{
        "best_estimator":SGDClassifier(loss='log', max_iter=2000, penalty='l1', random_state=state)
    },
    "Linear_Discriminant_Analysis":{
        "best_estimator":LinearDiscriminantAnalysis(store_covariance=True)
    },
    "K_Neighbors_Classifier":{
        "best_estimator":KNeighborsClassifier(algorithm='ball_tree', n_neighbors=9, weights='distance')
    },
    "Logistic_Regression":{
        "best_estimator":LogisticRegression(penalty='none', random_state=state, solver='saga')
    },
    "Decision_Tree_Classifier":{
        "best_estimator":DecisionTreeClassifier(criterion='entropy', random_state=state)
    },
    "SVC":{
        "best_estimator":SVC(C=10, gamma=0.1, probability=True, random_state=state)
    },
    "Nu_SVC":{
        "best_estimator":NuSVC(gamma=0.1, nu=0.2, probability=True, random_state=state)
    },
    "MLP_Classifier":{
        "best_estimator":MLPClassifier(alpha=0.05, max_iter=2000, random_state=state, solver='lbfgs')
    },
    "Random_Forest_Classifier":{
        "best_estimator":RandomForestClassifier(criterion='entropy', n_estimators=150, random_state=state)
    },
    "Gradient_Boosting_Classifier":{
        "best_estimator":GradientBoostingClassifier(learning_rate=0.15, max_depth=7,
                           min_samples_split=40, n_estimators=500,
                           random_state=state)
    }
}

### Extract Models

In [3]:
data = pd.read_csv('data/data_merged.csv')
oversample = SMOTE(random_state=3390)
year = 2019

for name, m in models.items():
    # Scenery 2 - Features
    games = data[['no_descriptors', 'strong_language', 'blood_and_gore', 'fantasy_violence', 'blood',
                          'mild_fantasy_violence', 'strong_sexual_content', 'sexual_themes', 'intense_violence',
                          'suggestive_themes', 'violence', 'simulated_gambling', 'sexual_content', 'language',
                          'mild_blood', 'mild_suggestive_themes', 'crude_humor', 'mild_violence', 'mild_lyrics',
                          'cartoon_violence', 'alcohol_reference', 'lyrics', 'drug_reference', 'use_of_alcohol',
                          'partial_nudity', 'nudity', 'mild_cartoon_violence', 'animated_blood', 'esrb_rating', 'release_year']]
    mapp = {'E':0, 'ET':1, 'T':2, 'M':3}
    games['esrb_rating'] = games['esrb_rating'].map(mapp)
    
    train = games[games['release_year'] < year]
    X_train, y_train = train.iloc[:,0:28], train.iloc[:,28]
    X_train, y_train = oversample.fit_resample(X_train, y_train)

    clf = m['best_estimator']
    clf.fit(X_train, y_train);
    joblib.dump(clf,f'models/{name}.pkl')
    
    # The following lines of code aren't needed for extract the models
    # It's only for veridyinf the new accuracy
    test = games[games['release_year'] >= year]
    X_test, y_test = test.iloc[:,0:28], test.iloc[:,28]
    print(f"{name:30} -> acc: {round(clf.score(X_test,y_test),5)}")

SGD_Classifier                 -> acc: 0.8037
Linear_Discriminant_Analysis   -> acc: 0.73889
K_Neighbors_Classifier         -> acc: 0.80185
Logistic_Regression            -> acc: 0.84259
Decision_Tree_Classifier       -> acc: 0.80556
SVC                            -> acc: 0.83704
Nu_SVC                         -> acc: 0.82593
MLP_Classifier                 -> acc: 0.85185
Random_Forest_Classifier       -> acc: 0.82037
Gradient_Boosting_Classifier   -> acc: 0.82778
